# Examples

The following examples show how to use the loader APIs:

## 1.1 Parsing a Workflow

This sample shows how to read a CWL document from a remote public URL:

In [ ]:
from cwl_loader import load_cwl_from_location

cwl_document = load_cwl_from_location('https://github.com/eoap/zarr-cloud-native-format/releases/download/0.3.0/app-water-bodies.0.3.0.cwl')

## 1.2 Serializing

This sample shows how to write a CWL document to a stream (string, file, ...):

In [ ]:
from cwl_loader import dump_cwl
import sys

dump_cwl(process=cwl_document, stream=sys.stderr)

## 1.3 Obtaining a dictionary

Probably users need a plain dictionary to add it inside another document, just invoke: 

In [ ]:
document_dict = dump_cwl(process=cwl_document)

import yaml
yaml.dump(
    document_dict,
    stream=sys.stdout,
    sort_keys=False
)

## 2.1 Parsing a CommandLineTool

This sample shows how to read a CWL document from a remote public URL:

In [ ]:
cwl_document = load_cwl_from_location('https://raw.githubusercontent.com/eoap/application-package-patterns/refs/heads/main/templates/stage-in-file.cwl')

## 2.2 Serializing

This sample shows how to write a CWL document to a stream (string, file, ...):

In [ ]:
dump_cwl(process=cwl_document, stream=sys.stderr)

## 3. Parsing existing data structures

Users can load CWL Worflow(s) even from existing dictionaries:

In [ ]:
from cwl_loader import load_cwl_from_yaml

cwl_document = load_cwl_from_yaml(raw_process={
  "cwlVersion": "v1.2", 
  "inputs": {
    "api_endpoint": {
      "doc": "STAC API endpoint for Landsat-9 data", 
      "type": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml#APIEndpoint", 
      "label": "STAC API endpoint"
    }, 
    "search_request": {
      "doc": "STAC API settings for Landsat-9 data", 
      "type": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/discovery.yaml#STACSearchSettings", 
      "label": "STAC API settings"
    }
  }, 
  "requirements": [
    {
      "class": "InlineJavascriptRequirement"
    }, 
    {
      "class": "NetworkAccess", 
      "networkAccess": True
    }, 
    {
      "class": "SchemaDefRequirement", 
      "types": [
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/string_format.yaml"
        }, 
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/geojson.yaml"
        }, 
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml"
        }, 
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/discovery.yaml"
        }
      ]
    }
  ], 
  "doc": "This tool uses the STAC Client to search for STAC items\n", 
  "class": "CommandLineTool", 
  "baseCommand": [
    "stac-client"
  ], 
  "label": "STAC Client Tool", 
  "arguments": [
    "search", 
    "$(inputs.api_endpoint.url.value)", 
    "${ const args = []; const collections = inputs.search_request.collections; args.push('--collections', collections.join(\",\")); return args; }", 
    "${ const args = []; const bbox = inputs.search_request?.bbox; if (Array.isArray(bbox) && bbox.length >= 4) { args.push('--bbox', ...bbox.map(String)); } return args; }", 
    "${ const args = []; const limit = inputs.search_request?.limit; args.push(\"--limit\", (limit ?? 10).toString()); return args; }", 
    "${ const maxItems = 5; return ['--max-items', maxItems.toString()]; }", 
    "${ const args = []; const filter = inputs.search_request?.filter; const filterLang = inputs.search_request?.['filter-lang']; if (filterLang) { args.push('--filter-lang', filterLang); } if (filter) { args.push('--filter', JSON.stringify(filter)); } return args; }", 
    "${ const datetime = inputs.search_request?.datetime; const datetimeInterval = inputs.search_request?.datetime_interval; if (datetime) { return ['--datetime', datetime]; } else if (datetimeInterval) { const start = datetimeInterval.start?.value || '..'; const end = datetimeInterval.end?.value || '..'; return ['--datetime', `${start}/${end}`]; } return []; }", 
    "${ const ids = inputs.search_request?.ids; const args = []; if (Array.isArray(ids) && ids.length > 0) { args.push('--ids', ...ids.map(String)); } return args; }", 
    "${ const intersects = inputs.search_request?.intersects; if (intersects) { return ['--intersects', JSON.stringify(intersects)]; } return []; }", 
    "--save", 
    "discovery-output.json"
  ], 
  "outputs": {
    "search_output": {
      "outputBinding": {
        "glob": "discovery-output.json"
      }, 
      "type": "File"
    }
  }, 
  "id": "stac-client", 
  "hints": [
    {
      "dockerPull": "docker.io/library/stac-client", 
      "class": "DockerRequirement"
    }
  ]
})

dump_cwl(process=cwl_document, stream=sys.stderr)

## 4. Parsing conditional Workflow

In [ ]:
cwl_document = load_cwl_from_location('https://raw.githubusercontent.com/eoap/how-to/refs/heads/main/cwl-workflows/conditional-workflows.cwl')
dump_cwl(process=cwl_document, stream=sys.stderr)